In [27]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle

In [61]:
# with open('twi22/processed/topic_tweet1_covid.pkl', 'rb') as f:
#     G = pickle.load(f)
# tweet = pd.read_csv('twi22/processed/topic_tweet1_covid_score.csv')

# with open('twi22/processed/topic_tweet2_war.pkl', 'rb') as f:
#     G = pickle.load(f)
# tweet = pd.read_csv('twi22/processed/topic_tweet2_war_score.csv')

with open('twi22/processed/topic_tweet3_climate.pkl', 'rb') as f:
    G = pickle.load(f)
tweet = pd.read_csv('twi22/processed/topic_tweet3_climate_score.csv')

tweet.set_index('uid', inplace=True)
print("Nodes: ", len(G.nodes()), "Len tweet: ", len(tweet))
print("Edges: ", len(G.edges()))

Nodes:  63865 Len tweet:  63865
Edges:  407927


In [62]:
# treat标-1，control标1，其余标0
def set_treat_label(G, focal_node):
    # return treat_label + propagate_label
    if tweet.loc[focal_node]['StScore']>0.8:
        prop = 1
    else:
        prop = 0
    if G.nodes[focal_node]['label']=='bot':
        return 0, prop
    following_nodes = list(G.successors(focal_node))
    condition = []
    for check_node_id in following_nodes:
        if tweet.loc[check_node_id]['StScore']>0.8:    # sentiment threshold
            if G.nodes[check_node_id]['label']=='human':
                condition.append(1)
            elif G.nodes[check_node_id]['label']=='bot':
                condition.append(2)
    exist_1 = 1 in set(condition)
    exist_2 = 2 in set(condition)
    exist_both = exist_1 and exist_2

    if exist_1 and not exist_both:
        return -1, prop # treat
    elif exist_2 and not exist_both:
        return 1, prop # control
    else:
        return 0, prop

i = 0
for node in G.nodes:
    i+=1
    G.nodes[node]['treat'], G.nodes[node]['prop'] = set_treat_label(G, node)
    if i%1000 == 0:
        print(i)
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000


In [63]:
node_map = {node: idx for idx, node in enumerate(G.nodes)}
# 有向图edge关系，第0行follow第1行
edge_index = np.array([[node_map[edge[0]], node_map[edge[1]]] for edge in G.edges])
print("Edges: ", edge_index.shape)

bot_label = np.array([0 if G.nodes[i]['label']=='human' else 1 for i in G.nodes])
treat_label = np.array([G.nodes[i]['treat'] for i in G.nodes])
prop_label = np.array([G.nodes[i]['prop'] for i in G.nodes])
y_truth = np.array([tweet.loc[i]['StScore'] for i in G.nodes])
print("b, t, prop, y shape: ", bot_label.shape, treat_label.shape, y_truth.shape, prop_label.shape)


Edges:  (407927, 2)
b, t, prop, y shape:  (63865,) (63865,) (63865,) (63865,)


In [64]:
# np.save('twi22/t1_covid/t1_edge.npy', edge_index)
# np.save('twi22/t1_covid/t1_bot_label.npy', bot_label)
# np.save('twi22/t1_covid/t1_T_label.npy', treat_label)
# np.save('twi22/t1_covid/t1_y.npy', y_truth)
# np.save('twi22/t1_covid/t1_prop_label.npy', prop_label)

# np.save('twi22/t2_war/t2_edge.npy', edge_index)
# np.save('twi22/t2_war/t2_bot_label.npy', bot_label)
# np.save('twi22/t2_war/t2_T_label.npy', treat_label)
# np.save('twi22/t2_war/t2_y.npy', y_truth)
# np.save('twi22/t2_war/t2_prop_label.npy', prop_label)

np.save('twi22/t3_climate/t3_edge.npy', edge_index)
np.save('twi22/t3_climate/t3_bot_label.npy', bot_label)
np.save('twi22/t3_climate/t3_T_label.npy', treat_label)
np.save('twi22/t3_climate/t3_y.npy', y_truth)
np.save('twi22/t3_climate/t3_prop_label.npy', prop_label)